<a href="https://colab.research.google.com/github/kobe4cn/modelscope_master/blob/main/%E5%9B%BE%E5%83%8F%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv_python
!pip install tf_slim
!pip install modelscope
!pip install pyperclip
!pip install pyclipper
!pip install openpyxl
!pip install xlwings

In [4]:
!pip install transformers

In [5]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import numpy as np
import cv2
import math
import pyclipper


2023-07-13 17:32:41,318 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-07-13 17:32:41,331 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-07-13 17:32:41,336 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-07-13 17:32:41,447 - modelscope - INFO - Loading done! Current index file version is 1.7.1, with md5 6110c6fa39783f3c1b4bb7e0d9b84fd0 and a total number of 861 components indexed


In [6]:
# 在图片上定位文本位置
def crop_image(img, position):
    def distance(x1,y1,x2,y2):
        return math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
    position = position.tolist()
    for i in range(4):
        for j in range(i+1, 4):
            if(position[i][0] > position[j][0]):
                tmp = position[j]
                position[j] = position[i]
                position[i] = tmp
    if position[0][1] > position[1][1]:
        tmp = position[0]
        position[0] = position[1]
        position[1] = tmp

    if position[2][1] > position[3][1]:
        tmp = position[2]
        position[2] = position[3]
        position[3] = tmp

    x1, y1 = position[0][0], position[0][1]
    x2, y2 = position[2][0], position[2][1]
    x3, y3 = position[3][0], position[3][1]
    x4, y4 = position[1][0], position[1][1]

    corners = np.zeros((4,2), np.float32)
    corners[0] = [x1, y1]
    corners[1] = [x2, y2]
    corners[2] = [x4, y4]
    corners[3] = [x3, y3]

    img_width = distance((x1+x4)/2, (y1+y4)/2, (x2+x3)/2, (y2+y3)/2)
    img_height = distance((x1+x2)/2, (y1+y2)/2, (x4+x3)/2, (y4+y3)/2)

    corners_trans = np.zeros((4,2), np.float32)
    corners_trans[0] = [0, 0]
    corners_trans[1] = [img_width - 1, 0]
    corners_trans[2] = [0, img_height - 1]
    corners_trans[3] = [img_width - 1, img_height - 1]

    transform = cv2.getPerspectiveTransform(corners, corners_trans)
    dst = cv2.warpPerspective(img, transform, (int(img_width), int(img_height)))
    return dst

In [7]:
def order_point(coor):
    arr = np.array(coor).reshape([4, 2])
    sum_ = np.sum(arr, 0)
    centroid = sum_ / arr.shape[0]
    theta = np.arctan2(arr[:, 1] - centroid[1], arr[:, 0] - centroid[0])
    sort_points = arr[np.argsort(theta)]
    sort_points = sort_points.reshape([4, -1])
    if sort_points[0][0] > centroid[0]:
        sort_points = np.concatenate([sort_points[3:], sort_points[:3]])
    sort_points = sort_points.reshape([4, 2]).astype('float32')
    return sort_points

In [ ]:
#先检测图片中的文本位置
ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-line-level_damo')
# ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-line-level_damo')
#ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-db-line-level_damo')
#基于文本位置，识别文本内容信息
ocr_recognition = pipeline(Tasks.ocr_recognition, model='damo/cv_convnextTiny_ocr-recognition-general_damo')
#文本属性识别
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-ecom-50cls')


In [9]:
import cv2
import numpy as np
#将文本信息绘制回图片
def draw_boxes_on_image(image_path, output_path, box_coordinates,label):

# 读取图片
    image = cv2.imread(image_path)

    # 将框的坐标绘制到图片上
    pts = np.array(box_coordinates, np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], isClosed=True, color=(0, 0, 255), thickness=1)  # 修改线宽为1

    # 在每个框的位置上增加序号标记
    label = label
    #cv2.putText(image, str(label), (int(box_coordinates[0]), int(box_coordinates[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

    cv2.putText(image, str(label), (int(box_coordinates[0]), int(box_coordinates[1])),cv2.FONT_HERSHEY_SIMPLEX | cv2.FONT_ITALIC, 0.6, (0, 0, 255), 2)


    # 保存修改后的图片
    cv2.imwrite(output_path, image)

In [10]:
import os
#遍历目录读取图片
def get_all_file_paths(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(os.path.abspath(file_path))
    return file_paths

# 示例用法
# directory = "/content/drive/MyDrive/Colab Notebooks/modelscope/images"
# file_paths = get_all_file_paths(directory)
# print(file_paths)

In [11]:
#增加输出后缀
import os

def add_out_suffix(file_path):
    dir_name = os.path.dirname(file_path)  # 获取文件所在目录路径
    file_name = os.path.basename(file_path)  # 获取文件名称
    file_name, extension = os.path.splitext(file_name)  # 将文件名称和扩展名拆分开

    # 在文件所在路径的同级目录下创建一个名为 "out" 的文件夹
    out_dir = os.path.join(os.path.dirname(dir_name), "out")
    os.makedirs(out_dir, exist_ok=True)  # 创建 "out" 文件夹，如果已存在则不进行任何操作

    # 在文件名称中添加 "_out" 后缀
    new_file_name = file_name + "_out" + extension

    # 构建新的文件路径
    new_file_path = os.path.join(out_dir, new_file_name)

    return new_file_path
add_out_suffix('/content/drive/MyDrive/Colab Notebooks/modelscope/images/lamer.jpg')

'/content/drive/MyDrive/Colab Notebooks/modelscope/out/lamer_out.jpg'

In [12]:
#将解析到图片信息分行存储
def save_text_to_string(saved_text,text):

    saved_text += text + "\n"  # 将每行文本追加到字符串，并添加换行符

    return saved_text

In [13]:
#输出的图片和抽取的图片文字内容转换成excel输出

import os
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

def append_to_excel(image_path, text, output_file):
    output_dir = os.path.dirname(output_file)  # 获取输出文件所在目录路径
    os.makedirs(output_dir, exist_ok=True)  # 创建输出目录，如果已存在则不进行任何操作

    # 检查输出文件是否存在
    file_exists = os.path.isfile(output_file)

    # 加载Excel文件或创建新的工作簿和工作表
    if file_exists:
        wb = Workbook()
        wb = openpyxl.load_workbook(output_file)
        sheet = wb.active
    else:
        wb = Workbook()
        sheet = wb.active
        sheet.append(['Image', 'Text','Product'])

    # 插入图片路径和文本信息到新行
    row = sheet.max_row + 1
    sheet['B' + str(row)].value = text
    sheet.column_dimensions['B'].width = 80

    # 插入图片到指定位置
    img = Image(image_path)
    sheet.add_image(img, 'A{}'.format(row))

    # 缩小图片尺寸
    img_width = img.width
    img_height = img.height
    # 调整单元格大小以适应缩小后的图片尺寸
    sheet.column_dimensions['A'].width = 80 #img.width / 6  # 调整列宽
    sheet.row_dimensions[row].height = img.height / 1.5  # 调整行高
    img.width = int(img_width * 0.8)
    img.height = int(img_height * 0.8)

    # # 调整单元格大小以适应缩小后的图片尺寸
    # sheet.column_dimensions['A'].width = img.width / 6  # 调整列宽
    # sheet.row_dimensions[row].height = img.height / 1.5  # 调整行高

    # 保存Excel文件
    wb.save(output_file)




In [14]:
import datetime
#日期格式文件名生成
def generate_file_name_with_date(extension):
    current_datetime = datetime.datetime.now()
    formatted_date = current_datetime.strftime("%Y-%m-%d")
    file_name = f"output_{formatted_date}.{extension}"
    return file_name

In [ ]:
#产品数据输出

import os
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

def append_product_excel(text, output_file):
    output_dir = os.path.dirname(output_file)  # 获取输出文件所在目录路径
    os.makedirs(output_dir, exist_ok=True)  # 创建输出目录，如果已存在则不进行任何操作

    # 检查输出文件是否存在
    file_exists = os.path.isfile(output_file)

    # 加载Excel文件或创建新的工作簿和工作表
    if file_exists:
        wb = Workbook()
        wb = openpyxl.load_workbook(output_file)
        sheet = wb.active
    else:
        wb = Workbook()
        sheet = wb.active
        sheet.append(['Image', 'Text','Product'])

    # 插入图片路径和文本信息到新行
    row = sheet.max_row + 1
    sheet['C' + str(row)].value = text


    # 保存Excel文件
    wb.save(output_file)

In [ ]:


import numpy as np

def box_center(box):
    """计算框的中心点坐标"""
    x = np.mean(box[::2])
    y = np.mean(box[1::2])
    return x, y

#找尺寸相关标签的最近列
def find_closest_above(boxes, target_box):
    """找出在给定框正上方且距离最近的框"""
    target_center = box_center(target_box)
    closest_box = None
    closest_center_dist = np.inf
    for box in boxes:
        center_x, center_y = box_center(box)
        if center_y < target_center[1]:  # 如果框在给定框的正上方
            dist = np.sqrt((center_x - target_center[0])**2 + (center_y - target_center[1])**2)  # 计算欧氏距离
            if dist < closest_center_dist:
                closest_center_dist = dist
                closest_box = box
    return closest_box



# 假设你的框在数组boxes中，你想找到在target_box上方的框
# boxes = np.array([472, 659, 524, 659, 524, 674, 472, 674])  # 你的框数组
# boxes = np.array([
#     [137, 649, 213, 648, 214, 657, 138, 658],
#     [473, 634, 525, 634, 525, 652, 473, 653],
#     [276, 662, 400, 662, 400, 684, 276, 684]

# ])
# target_box = np.array([474.0,659.0,524.0,659.0,524.0,673.0,474.0,673.0])  # 你的目标框
# closest_box = find_closest_above(boxes, target_box)
# print(closest_box)
# string = ', '.join(map(str, closest_box))
# print(dict_data[string])


In [ ]:
#解析所有商品增加进商品列
def find_product(output_file):
  # print(len(product_arr))
  # print(dict_data)
  for pro in product_arr:
    closest_box = find_closest_above(det_result, pro)
    # print(closest_box)
    close_str= ', '.join(map(str, closest_box))
    pro_int = [int(x) for x in pro]
    pro_str = ', '.join(map(str, pro_int))
    # print(pro_str)
    # print(dict_data[close_str])
    # print(dict_data[pro_str])
    dict_data[close_str]=dict_data[close_str]+dict_data[pro_str]
  # print(dict_data)
  str_result=''
  for value in dict_data.values():
    result= ner_pipeline(value)
    # print(result)
    check=0
    for item in result['output']:
      if '产品_核心产品' in item["type"]:
        check+=1
      if '尺寸规格' in item["type"]:
        check+=1
      if check>=2:
        str_result+=value+'\n'
        break
  append_product_excel(str_result,output_file)
  print(str_result,output_file)


In [ ]:
#获取目录下的所有需要解析的图片
image_files=get_all_file_paths('/content/drive/MyDrive/Colab Notebooks/modelscope/images')
#图片中解析的文本信息存储在这个字段

#文件输出路径
file_out='/content/drive/MyDrive/Colab Notebooks/modelscope/excel/'+""+generate_file_name_with_date('xlsx')
for img_path in image_files:
  text_parsing=""
  #图片路径或者提供URL
  img_path = img_path
  #转换未来图片的输出路径
  out_path = add_out_suffix(img_path)
  image_full = cv2.imread(img_path)
  #图片输出路径
  cv2.imwrite(out_path, image_full)
  # #也可以提供URL
  # img_url='https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/ocr_detection.jpg'

  #图片位置
  det_result = ocr_detection(image_full)

  det_result = det_result['polygons']
  dict_data={}
  product_arr=[]
  for i in range(det_result.shape[0]):
      pts = order_point(det_result[i])
      #图片位置
      image_crop = crop_image(image_full, pts)
      result = ocr_recognition(image_crop)
      #print(list(pts.reshape(-1)))
      # 读取图片
      image = cv2.imread(out_path)

      # 定义文本信息和框的坐标
      box_coordinates = list(pts.reshape(-1))

      # 将文本信息和框的坐标绘制到图片上
      draw_boxes_on_image(out_path, out_path, box_coordinates,i+1)

      print("box: %s" % ', '.join([str(int(e)) for e in list(pts.reshape(-1))]))

      print("text: %s. %s" % ((i+1),result['text']))

      #识别出来的文本内容
      word_content="".join(result['text'])
      #框的位置
      frame_location=', '.join([str(int(e)) for e in list(pts.reshape(-1))])
      #识别文本属性
      # result = ner_pipeline('云绒霜')
      if word_content !='':
        result= ner_pipeline(word_content)
        #print(result)
        if len(result['output'])==1 and '尺寸规格' in result['output'][0]['type']:
          product_arr.append(list(pts.reshape(-1)))
      dict_data[frame_location]=word_content
      text_parsing = save_text_to_string(text_parsing, str(i+1) + "." + word_content)
     #text_parsing=save_text_to_string(text_parsing,str(i+1)+"."+result['text'])

  append_to_excel(out_path,text_parsing,file_out)
  find_product(file_out)



*加入产品的新列*


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [54]:
def find_same_column_and_vertical_nearby_boxes(boxes, column_threshold, vertical_threshold):
    same_column_boxes = []
    nearby_boxes = {'vertical': []}
    num_boxes = len(boxes)

    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            box1 = boxes[i]
            box2 = boxes[j]

            # 判断框1和框2是否在同一列
            if abs(box1[0] - box2[0]) <= column_threshold:
                # 判断框1和框2是否在垂直方向上邻近
                if abs(box1[1] - box2[1]) <= vertical_threshold or abs(box1[3] - box2[3]) <= vertical_threshold:
                    same_column_boxes.append((i, j))
                    nearby_boxes['vertical'].append((i, j))

    return same_column_boxes, nearby_boxes

boxes = [

    [472, 659, 524, 659, 524, 674, 472, 674],

    [468, 634, 525, 634, 525, 653, 468, 653]
]

column_threshold = 10  # 判断同一列的阈值
vertical_threshold = 10  # 判断上下邻近的阈值

same_column_boxes, nearby_boxes_vertical = find_same_column_and_vertical_nearby_boxes(
    boxes, column_threshold, vertical_threshold
)

print("Same column and vertical nearby boxes:")
for pair in same_column_boxes:
    print(boxes[pair[0]], boxes[pair[1]])

print("\nVertical nearby boxes:")
for pair in nearby_boxes_vertical['vertical']:
    print(boxes[pair[0]], boxes[pair[1]])


Same column and vertical nearby boxes:

Vertical nearby boxes:


In [40]:
det_result

#dict_data

array([[532, 777, 729, 779, 728, 795, 530, 792],
       [ 67, 778, 536, 778, 536, 793,  67, 793],
       [276, 662, 400, 662, 400, 684, 276, 684],
       [472, 659, 524, 659, 524, 674, 472, 674],
       [688, 657, 739, 657, 739, 676, 688, 676],
       [584, 658, 620, 658, 620, 676, 584, 676],
       [136, 646, 229, 646, 229, 661, 136, 661],
       [685, 633, 740, 633, 740, 652, 685, 652],
       [573, 634, 627, 634, 627, 653, 573, 653],
       [468, 634, 525, 634, 525, 653, 468, 653],
       [130, 634, 202, 637, 201, 648, 129, 645],
       [560, 610, 634, 610, 634, 625, 560, 625],
       [698, 602, 728, 602, 728, 613, 698, 613],
       [580, 601, 620, 601, 620, 613, 580, 613],
       [493, 571, 524, 571, 524, 582, 493, 582],
       [460, 571, 489, 571, 489, 582, 460, 582],
       [357, 554, 372, 554, 372, 612, 357, 612],
       [663, 503, 758, 503, 758, 517, 663, 517],
       [124, 488, 226, 488, 226, 511, 124, 511],
       [357, 483, 373, 483, 373, 523, 357, 523],
       [668, 473, 75

In [38]:
import math

def calculate_center(box):
    x1, y1, x2, y2, x3, y3, x4, y4 = box
    center_x = (x1 + x3) / 2
    center_y = (y1 + y3) / 2
    return center_x, center_y


def is_same_row(box1, box2, threshold):
    center1 = calculate_center(box1)
    center2 = calculate_center(box2)
    distance = math.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)
    return abs(center1[1] - center2[1]) <= threshold and distance <= threshold


def find_same_row_boxes(boxes, threshold):
    same_row_boxes = []
    num_boxes = len(boxes)

    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            box1 = boxes[i]
            box2 = boxes[j]
            if is_same_row(box1, box2, threshold):
                same_row_boxes.append((i, j))

    return same_row_boxes


def find_nearby_column_boxes(boxes, threshold):
    nearby_column_boxes = []
    num_boxes = len(boxes)

    for i in range(num_boxes):
        for j in range(i + 1, num_boxes):
            box1 = boxes[i]
            box2 = boxes[j]
            if abs(box1[0] - box2[0]) <= threshold:  # 判断 x 坐标差值是否小于阈值
                nearby_column_boxes.append((i, j))

    return nearby_column_boxes




same_row_threshold = 20  # 判断同一行的阈值
nearby_column_threshold = 5  # 判断上下具体很近的同一列的阈值

same_row_boxes = find_same_row_boxes(det_result, same_row_threshold)
nearby_column_boxes = find_nearby_column_boxes(det_result, nearby_column_threshold)




print("Same row boxes:")
for row in same_row_boxes:
    print(det_result[row[0]])

print("\nNearby column boxes:")
for col in nearby_column_boxes:
    print(det_result[col[0]], det_result[col[1]])



Same row boxes:
[560 610 634 610 634 625 560 625]
[493 329 564 329 564 340 493 340]
[588 324 679 324 679 342 588 342]

Nearby column boxes:
[276 662 400 662 400 684 276 684] [271 333 323 333 323 348 271 348]
[472 659 524 659 524 674 472 674] [468 634 525 634 525 653 468 653]
[688 657 739 657 739 676 688 676] [685 633 740 633 740 652 685 652]
[584 658 620 658 620 676 584 676] [580 601 620 601 620 613 580 613]
[584 658 620 658 620 676 584 676] [588 324 679 324 679 342 588 342]
[584 658 620 658 620 676 584 676] [589 307 678 307 678 322 589 322]
[584 658 620 658 620 676 584 676] [583 282 686 278 687 301 584 304]
[580 601 620 601 620 613 580 613] [583 282 686 278 687 301 584 304]
[493 571 524 571 524 582 493 582] [493 329 564 329 564 340 493 340]
[357 554 372 554 372 612 357 612] [357 483 373 483 373 523 357 523]
[663 503 758 503 758 517 663 517] [668 473 753 473 753 491 668 491]
[588 324 679 324 679 342 588 342] [589 307 678 307 678 322 589 322]
[588 324 679 324 679 342 588 342] [583 282 6

In [39]:
det_result

array([[532, 777, 729, 779, 728, 795, 530, 792],
       [ 67, 778, 536, 778, 536, 793,  67, 793],
       [276, 662, 400, 662, 400, 684, 276, 684],
       [472, 659, 524, 659, 524, 674, 472, 674],
       [688, 657, 739, 657, 739, 676, 688, 676],
       [584, 658, 620, 658, 620, 676, 584, 676],
       [136, 646, 229, 646, 229, 661, 136, 661],
       [685, 633, 740, 633, 740, 652, 685, 652],
       [573, 634, 627, 634, 627, 653, 573, 653],
       [468, 634, 525, 634, 525, 653, 468, 653],
       [130, 634, 202, 637, 201, 648, 129, 645],
       [560, 610, 634, 610, 634, 625, 560, 625],
       [698, 602, 728, 602, 728, 613, 698, 613],
       [580, 601, 620, 601, 620, 613, 580, 613],
       [493, 571, 524, 571, 524, 582, 493, 582],
       [460, 571, 489, 571, 489, 582, 460, 582],
       [357, 554, 372, 554, 372, 612, 357, 612],
       [663, 503, 758, 503, 758, 517, 663, 517],
       [124, 488, 226, 488, 226, 511, 124, 511],
       [357, 483, 373, 483, 373, 523, 357, 523],
       [668, 473, 75

In [19]:
# 计算每个框的中心点坐标
# 定义阈值
thresh = 20
edges = np.zeros((det_result.shape[0], 4))
for i, box in enumerate(det_result):
    left = box[0]
    top = box[1]
    right = box[4]
    bottom = box[5]
    edges[i] = [left, top, right, bottom]

# 对于每对框，如果它们的边缘中点在x或y方向上的距离小于阈值，则认为它们在同一行或列
same_row = []
same_column = []
for i in range(len(edges)):
    for j in range(i + 1, len(edges)):
        # 在同一行的条件是上下边缘相近
        if abs(edges[i, 1] - edges[j, 1]) < thresh and abs(edges[i, 3] - edges[j, 3]) < thresh:
            same_row.append((i, j))
        # 在同一列的条件是左右边缘相近
        if abs(edges[i, 0] - edges[j, 0]) < thresh and abs(edges[i, 2] - edges[j, 2]) < thresh:
            same_column.append((i, j))

print('在同一行的框的索引对：', same_row)
print('在同一列的框的索引对：', same_column)

在同一行的框的索引对： [(1, 2), (1, 3), (2, 3), (7, 8), (11, 12), (11, 13), (11, 14), (12, 13), (12, 14), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (15, 16), (15, 17), (15, 18), (15, 19), (16, 17), (16, 18), (16, 19), (17, 18), (17, 19), (18, 19), (21, 22), (24, 25), (29, 30), (32, 33), (37, 38), (43, 44), (62, 63), (62, 64), (62, 65), (63, 64), (63, 65), (64, 65), (66, 67), (68, 69), (70, 71), (70, 72), (71, 72), (73, 74), (76, 77), (76, 78), (76, 79), (77, 78), (77, 79), (78, 79), (80, 81), (80, 82), (80, 83), (81, 82), (81, 83), (82, 83), (84, 85), (84, 86), (84, 87), (84, 88), (85, 86), (85, 87), (85, 88), (86, 87), (86, 88), (87, 88), (89, 90), (89, 91), (89, 92), (89, 93), (90, 91), (90, 92), (90, 93), (91, 92), (91, 93), (92, 93), (94, 95), (94, 96), (94, 97), (94, 98), (95, 96), (95, 97), (95, 98), (96, 97), (96, 98), (97, 98), (99, 100), (99, 101), (99, 102), (99, 103), (100, 101), (100, 102), (100, 103), (101, 102), (10

In [19]:
#判断两个框是不是在同一行
def are_boxes_in_same_row(box1, box2, threshold):
    box1_coords = [float(coord) for coord in box1.split(',')]
    box2_coords = [float(coord) for coord in box2.split(',')]

    # 提取框的 y 坐标
    y1_box1 = box1_coords[1]
    y1_box2 = box2_coords[1]

    # 比较框的 y 坐标并判断是否在同一行
    threshold = 5  # 阈值，用于判断两个框在 y 坐标上的差异

    if abs(y1_box1 - y1_box2) <= threshold:
        print("两个框在同一行")
    else:
        print("两个框不在同一行")

#判断哪个框在上，哪个在下
def are_frist_box_is_front(box1,box2):
  # 解析框的坐标信息
    box1_coords = [float(coord) for coord in box1.split(',')]
    box2_coords = [float(coord) for coord in box2.split(',')]

    # 提取框的左边 x 坐标
    x1_box1 = box1_coords[0]
    x1_box2 = box2_coords[0]

    # 判断框的前后顺序
    if x1_box1 < x1_box2:
        print("框1在前，框2在后")
    elif x1_box1 > x1_box2:
        print("框2在前，框1在后")
    else:
        print("框1和框2重叠或位置相同")

#判断两个框是不是在同一列
# 解析框的坐标信息
def are_boxes_in_same_col(box1,box2):
    # 解析框的坐标信息
    box1_coords = [float(coord) for coord in box1.split(',')]
    box2_coords = [float(coord) for coord in box2.split(',')]

    # 提取框的左边和右边 x 坐标
    x1_box1 = box1_coords[0]
    x2_box1 = box1_coords[2]
    x1_box2 = box2_coords[0]
    x2_box2 = box2_coords[2]

    # 设置接近阈值
    threshold = 5  # 阈值，用于判断两个框在 x 坐标上的接近程度

    # 判断框是否在同一列并且非常接近
    if abs(x1_box1 - x1_box2) <= threshold and abs(x2_box1 - x2_box2) <= threshold:
        print("两个框在同一列并且非常接近")
    else:
        print("两个框不在同一列或不够接近")
584.0,658.0,620.0,658.0,620.0,676.0,584.0,676.0
text: 6. ['7ml']
box: 136.0,646.0,229.0,646.0,229.0,661.0,136.0,661.0
text: 7. ['LA LOTION PRE SOM']
box: 685.0,633.0,740.0,633.0,740.0,652.0,685.0,652.0
text: 8. ['云绒霜']
box: 573.0,634.0,627.0,634.0,627.0,653.0,573.0,653.0
text: 9. ['云绒霜']
are_boxes_in_same_col('584.0,658.0,620.0,658.0,620.0,676.0,584.0,676.0','573.0,634.0,627.0,634.0,627.0,653.0,573.0,653.0')
# are_boxes_in_same_row('532.0,777.0,729.0,779.0,728.0,795.0,530.0,792.0','67.0,778.0,536.0,778.0,536.0,793.0,67.0,793.0',5)
# are_frist_box_is_front('532.0,777.0,729.0,779.0,728.0,795.0,530.0,792.0','67.0,778.0,536.0,778.0,536.0,793.0,67.0,793.0')

两个框不在同一列或不够接近


In [55]:
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-ecom-50cls')


2023-07-13 11:35:01,403 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
Downloading: 100%|██████████| 10.9k/10.9k [00:00<00:00, 15.2MB/s]
Downloading: 100%|██████████| 175/175 [00:00<00:00, 588kB/s]
Downloading: 100%|██████████| 52.0k/52.0k [00:00<00:00, 122kB/s]
Downloading: 100%|██████████| 391M/391M [00:28<00:00, 14.6MB/s]
Downloading: 100%|██████████| 7.68k/7.68k [00:00<00:00, 8.63MB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 406kB/s]
Downloading: 100%|██████████| 429k/429k [00:01<00:00, 417kB/s]
Downloading: 100%|██████████| 364/364 [00:00<00:00, 1.94MB/s]
Downloading: 100%|██████████| 107k/107k [00:00<00:00, 163kB/s]
2023-07-13 11:35:39,342 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/nlp_raner_named-entity-recognition_chinese-base-ecom-50cls
2023-07-13 11:35:39,343 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/nlp_raner_named-entity-recognition_chinese-base-ecom-50